In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Mount với drive, cái này HH mount với drive của HH, mà drive của HH thì có tạo shortcut đến thư mục "Vietnam Datathon" của Vũ Lêm
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set working path nè
working_path = '/content/drive/MyDrive/Vietnam Datathon/2) Data/2.1) Raw Data/Provided Data/'
df1 = pd.read_csv(working_path + 'train.csv')
df1

In [ ]:
df1.info()

Cột `Date` đang ở kiểu `object` (dạng text), không phải dạng thời gian.

In [ ]:
df1.describe()

Cột `Weekly_Sales` có giá trị `min` (nhỏ nhất) là **-4988.94**.

In [ ]:
(df1['Weekly_Sales'] < 0).sum()

Có tới **1285** hàng dữ liệu bị âm ("Trả hàng").

In [ ]:
df1.nunique()

In [ ]:
# 1. Đọc dữ liệu
df = pd.read_csv(working_path + 'train.csv')

# 2. Xem tổng quan dữ liệu
print("Kích thước dữ liệu:", df.shape)
print("\n5 dòng đầu:")
print(df.head())
print("\nThông tin cột và kiểu dữ liệu:")
print(df.info())
print("\nThống kê mô tả:")
print(df.describe())

# 3. Kiểm tra giá trị thiếu
print("\nGiá trị thiếu trong từng cột:")
print(df.isnull().sum())

# 4. Kiểm tra dữ liệu trùng lặp
print("\nSố bản ghi trùng lặp:", df.duplicated().sum())

# 5. Chuyển đổi kiểu dữ liệu (nếu cần)
df['Date'] = pd.to_datetime(df['Date'])  # Chuyển cột Date sang datetime
df['IsHoliday'] = df['IsHoliday'].astype(bool)  # Chuyển IsHoliday sang boolean

# 6. Kiểm tra phạm vi thời gian
print(f"\nPhạm vi thời gian: từ {df['Date'].min()} đến {df['Date'].max()}")

# Thêm cột thời gian để phân tích sâu hơn
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Week'] = df['Date'].dt.isocalendar().week



- **Kích thước:** 4,420 bản ghi, 5 cột

- **Thời gian:** Từ 05/02/2010 đến 26/10/2012

- Không có giá trị thiếu hoặc trùng lặp

In [ ]:
# 6. Phân tích phân phối của Weekly_Sales
plt.figure(figsize=(12, 6))
sns.histplot(df['Weekly_Sales'], bins=100, kde=True)
plt.title('Phân phối của Weekly_Sales')
plt.show()


Phân phối lệch phải rõ rệt

In [ ]:
# 7. Phân tích theo thời gian - CẢI TIẾN HIỂN THỊ THÁNG
plt.figure(figsize=(15, 10))

# Biểu đồ doanh thu theo tháng với trục ngang rõ ràng
plt.subplot(2, 2, 3)
monthly_sales = df.set_index('Date')['Weekly_Sales'].resample('M').mean()

# Tạo format cho trục ngang
plt.plot(monthly_sales.index, monthly_sales.values, marker='o', linewidth=2, markersize=4)
plt.title('Doanh thu trung bình theo tháng', fontsize=14, fontweight='bold')
plt.ylabel('Weekly Sales ($)', fontsize=12)
plt.xlabel('Thời gian', fontsize=12)
plt.grid(True, alpha=0.3)

# Định dạng trục ngang để hiển thị rõ tháng/năm
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%m/%Y'))
plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.MonthLocator(interval=3))  # Hiển thị mỗi 3 tháng
plt.xticks(rotation=45)

# Hiển thị biểu đồ
plt.tight_layout()
plt.show()

In [ ]:
# PHÂN TÍCH THEO THỜI GIAN CHI TIẾT
plt.figure(figsize=(16, 12))

# Biểu đồ 1: Doanh thu theo tháng
plt.subplot(2, 2, 1)
monthly_sales = df.set_index('Date')['Weekly_Sales'].resample('M').mean()
plt.plot(monthly_sales.index, monthly_sales.values,
         color='blue', linewidth=2.5, marker='o', markersize=4)
plt.title('DOANH THU TRUNG BÌNH THEO THÁNG', fontsize=14, fontweight='bold', pad=20)
plt.ylabel('Doanh thu ($)', fontsize=12)
plt.xlabel('Thời gian', fontsize=12)
plt.grid(True, alpha=0.3)

# Định dạng trục thời gian
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%m/%Y'))
plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.MonthLocator(interval=2))
plt.xticks(rotation=45)

# Biểu đồ 1: So sánh năm
plt.subplot(2, 2, 2)
yearly_comparison = df.groupby(['Year', 'Month'])['Weekly_Sales'].mean().unstack('Year')
yearly_comparison.plot(kind='line', marker='o', ax=plt.gca())
plt.title('SO SÁNH DOANH THU THEO THÁNG GIỮA CÁC NĂM', fontsize=14, fontweight='bold', pad=20)
plt.ylabel('Doanh thu ($)', fontsize=12)
plt.xlabel('Tháng', fontsize=12)
plt.legend(title='Năm')
plt.grid(True, alpha=0.3)


# THỐNG KÊ THEO THÁNG CHI TIẾT
print("\n" + "="*60)
print("THỐNG KÊ DOANH THU THEO THÁNG")
print("="*60)

monthly_stats = df.groupby('Month')['Weekly_Sales'].agg(['mean', 'std', 'min', 'max'])
monthly_stats.columns = ['Trung bình', 'Độ lệch chuẩn', 'Tối thiểu', 'Tối đa']
print(monthly_stats.round(2))

# Tìm tháng có doanh thu cao nhất và thấp nhất
best_month = monthly_stats['Trung bình'].idxmax()
worst_month = monthly_stats['Trung bình'].idxmin()

print(f"\n📈 THÁNG TỐT NHẤT: Tháng {best_month} - Doanh thu trung bình: ${monthly_stats.loc[best_month, 'Trung bình']:,.2f}")
print(f"📉 THÁNG THẤP NHẤT: Tháng {worst_month} - Doanh thu trung bình: ${monthly_stats.loc[worst_month, 'Trung bình']:,.2f}")

1. **Đỉnh 1: Mùa lễ cuối năm (Tháng 11 & 12)**
  - **Nhận xét:** Thấy hai cột cao nhất là Tháng 11 (Nov) và Tháng 12 (Dec).

  - **Ý nghĩa:** Đây chính là mùa mua sắm Lễ Tạ ơn (Thanksgiving - cuối T11) và Giáng sinh (Christmas - T12). Điều này khẳng định 100% rằng việc chúng ta xây dựng các feature "đếm ngược" (`countdown`) và "cửa sổ" (`window`) cho 2 lễ này (như `is_pre_christmas_window_week`) là bắt buộc.

2. **Đỉnh 2: "Cú hích" Hoàn thuế (Tháng 2, 3)**
  - **Nhận xét:** Đây mới là insight "độc đáo" nhất. Tháng 1 (Jan) là tháng "sụt giảm" thê thảm (do "holiday hangover" - mệt mỏi sau lễ). Nhưng ngay lập tức, doanh số trung bình tăng vọt trở lại và duy trì ở mức KHÁ CAO trong suốt Tháng 2 và 3

  - **Ý nghĩa (Liên kết với "Payday Pulse"):**

    + **Câu hỏi:** *Điều gì khiến 3 tháng này cao như vậy? Đó không phải là Lễ (Holiday) theo lịch của Walmart.*

    + Đây chính xác là "Mùa Hoàn thuế" (Tax Refund Season) của Mỹ mà chúng ta đã giả định (thường từ giữa T2 đến giữa T4). Đây chính là "Lương 13" hay "Thưởng Tết" của họ.

    + Biểu đồ này CHỨNG MINH rằng giả định của chúng ta là đúng: Các "cơn sóng" thu nhập (income pulse) có tác động cực kỳ lớn đến doanh thu, ngang ngửa với mùa Giáng sinh.

3. Các "Vùng trũng" (Tháng 1 & 9)
  - **Nhận xét:** Tháng 1 (Jan) và Tháng 9 (Sep) là các tháng thấp nhất.

  - **Ý nghĩa:**

    - **Tháng 1:** Khẳng định sự sụt giảm nghiêm trọng sau khi mùa lễ kết thúc.

    - **Tháng 9:** Dường như là tháng "lặng" nhất trong năm, không có lễ lớn, không có "sóng" thu nhập, và mùa "Back-to-School" (tựu trường) có vẻ không tạo ra cú hích đủ lớn trên toàn bộ hệ thống.


In [ ]:
# Phân tích Doanh thu theo tháng
monthly_sales = df.groupby('Month')['Weekly_Sales'].mean()
plt.figure(figsize=(10, 6))
monthly_sales.plot(kind='bar')
plt.title('Doanh thu trung bình theo Tháng')
plt.xlabel('Tháng')
plt.ylabel('Doanh thu trung bình')
plt.show()


In [ ]:
# Phân tích doanh thu theo ngày trong tháng
df['day_of_month'] = df['Date'].dt.day
df['is_payday_period'] = ((df['day_of_month'] >= 25) | (df['day_of_month'] <= 5)).astype(int)

plt.figure(figsize=(12, 6))
daily_pattern = df.groupby('day_of_month')['Weekly_Sales'].mean()
plt.plot(daily_pattern.index, daily_pattern.values, marker='o', linewidth=2)
plt.axvspan(27, 31, alpha=0.2, color='red', label='Cuối tháng (Payday)')
plt.axvspan(1, 4, alpha=0.2, color='orange', label='Đầu tháng (Payday)')
plt.title('CHU KỲ DOANH THU THEO NGÀY TRONG THÁNG', fontweight='bold')
plt.xlabel('Ngày trong tháng')
plt.ylabel('Doanh thu trung bình')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:

# 8. Phân tích theo ngày lễ
holiday_sales = df.groupby('IsHoliday')['Weekly_Sales'].mean()
print("\nDoanh thu trung bình theo ngày lễ:")
print(holiday_sales)

In [ ]:
# 9. Phân tích theo ngày lễ chi tiết
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
holiday_sales = df.groupby('IsHoliday')['Weekly_Sales'].mean()
colors = ['lightblue', 'red']
plt.pie(holiday_sales.values, labels=['Non-Holiday', 'Holiday'],
        autopct='%1.1f%%', colors=colors)
plt.title('Doanh thu trung bình theo ngày lễ')

plt.subplot(1, 2, 2)
sns.boxplot(data=df, x='IsHoliday', y='Weekly_Sales')
plt.title('Phân phối doanh thu theo ngày lễ')
plt.xlabel('Is Holiday')
plt.ylabel('Weekly Sales')

plt.tight_layout()
plt.show()

print("\n" + "="*50)
print("PHÂN TÍCH THEO NGÀY LỄ")
print("="*50)
print("Doanh thu trung bình theo ngày lễ:")
print(holiday_sales)
print(f"\nChênh lệch: {holiday_sales[True] - holiday_sales[False]:.2f}")